In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Initialize video capture
cap = cv2.VideoCapture('volleyball_match.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

# Initialize deque to store ball positions for trajectory
ball_positions = deque(maxlen=50)  # Store last 50 positions

# Initialize Kalman filter for ball tracking
kalman = cv2.KalmanFilter(4, 2)  # 4 state variables, 2 measurement variables
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03
measurement = np.array((2, 1), np.float32)
prediction = np.zeros((2, 1), np.float32)

# Variables to track ball detection
frames_without_detection = 0
max_frames_to_predict = 10

# Process video frame by frame
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    
    # Run YOLOv8 inference on the frame
    results = model(frame)
    
    # Process detections
    ball_detected = False
    left_players = 0
    right_players = 0
    
    for r in results:
        boxes = r.boxes
        
        for box in boxes:
            # Get box data
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Class 0 is person in COCO dataset
            if cls_id == 0 and conf > 0.5:
                center_x = (x1 + x2) // 2
                
                # Count players on left vs right side
                if center_x < width // 2:
                    left_players += 1
                    # Draw person bounding box with team color (blue)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                else:
                    right_players += 1
                    # Draw person bounding box with team color (green)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Class 32 is sports ball in COCO dataset
            elif cls_id == 32 and conf > 0.5:
                # Extract the ball region to check if it's white
                ball_roi = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]
                
                if ball_roi.size > 0:  # Make sure ROI is not empty
                    # Convert to HSV for better color filtering
                    hsv_roi = cv2.cvtColor(ball_roi, cv2.COLOR_BGR2HSV)
                    
                    # Define range for white color in HSV
                    lower_white = np.array([0, 0, 180])
                    upper_white = np.array([180, 40, 255])
                    
                    # Create mask for white pixels
                    white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                    white_ratio = cv2.countNonZero(white_mask) / (ball_roi.shape[0] * ball_roi.shape[1])
                    
                    # If a significant portion is white, consider it our volleyball
                    if white_ratio > 0.4:
                        ball_detected = True
                        center_x = (x1 + x2) // 2
                        center_y = (y1 + y2) // 2
                        
                        # Update measurement for Kalman filter
                        measurement = np.array([[np.float32(center_x)], [np.float32(center_y)]])
                        
                        # Correct Kalman filter with new measurement
                        kalman.correct(measurement)
                        
                        # Add ball position to trajectory
                        ball_positions.append((center_x, center_y))
                        
                        # Draw ball bounding box
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 165, 255), 2)
                        cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
                        
                        frames_without_detection = 0
    
    # If ball not detected, use Kalman filter to predict position
    if not ball_detected:
        frames_without_detection += 1
        
        if frames_without_detection < max_frames_to_predict and len(ball_positions) > 0:
            # Predict next position
            prediction = kalman.predict()
            pred_x = int(prediction[0, 0])
            pred_y = int(prediction[1, 0])
            
            # Only add prediction if it's within frame boundaries
            if 0 <= pred_x < width and 0 <= pred_y < height:
                ball_positions.append((pred_x, pred_y))
                # Draw predicted position with different color
                cv2.circle(frame, (pred_x, pred_y), 5, (255, 0, 255), -1)
    
    # Draw ball trajectory
    if len(ball_positions) > 1:
        for i in range(1, len(ball_positions)):
            thickness = int(np.sqrt(50 / float(i + 1)) * 2)
            cv2.line(frame, ball_positions[i-1], ball_positions[i], (0, 0, 255), thickness)
    
    # Draw court dividing line
    cv2.line(frame, (width // 2, 0), (width // 2, height), (255, 255, 255), 2)
    
    # Display player counts
    cv2.putText(frame, f"Left team: {left_players}", (20, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    cv2.putText(frame, f"Right team: {right_players}", (width - 220, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Write the processed frame to output video
    out.write(frame)
    
    # Optional: Display the frame (comment out for faster processing)
    cv2.imshow('Volleyball Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Processing complete. Output saved to 'output.avi'")


0: 384x640 18 persons, 43.9ms
Speed: 3.3ms preprocess, 43.9ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 41.7ms
Speed: 1.1ms preprocess, 41.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 41.4ms
Speed: 1.1ms preprocess, 41.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 44.9ms
Speed: 1.2ms preprocess, 44.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 44.8ms
Speed: 2.7ms preprocess, 44.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 41.3ms
Speed: 1.7ms preprocess, 41.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 41.5ms
Speed: 1.2ms preprocess, 41.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 42.5ms
Speed: 1.6ms preprocess, 42.5ms inference, 0.7ms postprocess per image at

  Using cached ultralytics-8.3.155-py3-none-any.whl.metadata (37 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 18.2 MB/s eta 0:00:0000:0100:01
Using cached ultralytics-8.3.155-py3-none-any.whl (1.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 18.6 MB/s eta 0:00:00a 0:00:01
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.3 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using ca

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque
import time

# Load YOLOv8 model with optimized parameters
model = YOLO('yolov8x.pt')

# Initialize video capture
cap = cv2.VideoCapture('volleyball_match.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer - using H264 for better performance
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

# Ball tracking parameters
ball_positions = deque(maxlen=30)  # Store recent positions
previous_frame = None  # For frame differencing
ball_size_range = (5, 30)  # Adjusted smaller min size for Olympic volleyball

# Process video frame by frame - optimize for speed
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    processing_start = time.time()  # Track processing time
    
    # STEP 1: FAST BALL DETECTION USING FRAME DIFFERENCING
    # Convert frame to grayscale for faster processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # If we have a previous frame, use frame differencing to find motion
    ball_candidates = []
    if previous_frame is not None:
        # Frame difference to detect movement
        frame_diff = cv2.absdiff(previous_frame, gray)
        _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Check each contour
        for contour in contours:
            # Small contours could be the ball
            if 20 < cv2.contourArea(contour) < 500:  # Small area for ball
                (x, y, w, h) = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                
                # Ball should be roughly circular
                if 0.8 < aspect_ratio < 1.2:
                    center_x = x + w // 2
                    center_y = y + h // 2
                    roi = frame[max(0, y):min(height, y+h), max(0, x):min(width, x+w)]
                    
                    if roi.size > 0:
                        # Check if this region is predominantly white (volleyball)
                        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                        lower_white = np.array([0, 0, 150])
                        upper_white = np.array([180, 50, 255])
                        white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                        white_ratio = cv2.countNonZero(white_mask) / (roi.shape[0] * roi.shape[1])
                        
                        if white_ratio > 0.3:  # If region is sufficiently white
                            ball_candidates.append({
                                'center': (center_x, center_y),
                                'size': max(w, h),
                                'white_ratio': white_ratio,
                                'score': white_ratio * (1 - abs(1 - aspect_ratio))  # Higher for circular white objects
                            })
    
    # Update previous frame
    previous_frame = gray.copy()
    
    # STEP 2: RUN YOLO DETECTION IN PARALLEL FOR PEOPLE
    results = model.predict(frame, conf=0.5, classes=[0])  # Only detect people (class 0) for speed
    
    # Count players on each side
    left_players = 0
    right_players = 0
    mid_line = width // 2
    
    for r in results:
        boxes = r.boxes
        
        for box in boxes:
            # Get box data
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x = (x1 + x2) // 2
            
            # Count players based on position
            if center_x < mid_line:
                left_players += 1
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue for left team
            else:
                right_players += 1
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green for right team
    
    # STEP 3: PROCESS BALL CANDIDATES
    if ball_candidates:
        # Sort by score
        ball_candidates.sort(key=lambda x: x['score'], reverse=True)
        best_candidate = ball_candidates[0]
        
        # Get ball position
        ball_x, ball_y = best_candidate['center']
        
        # Add ball position to trajectory
        ball_positions.append((ball_x, ball_y))
        
        # Draw ball position - make it more visible
        cv2.circle(frame, (ball_x, ball_y), 10, (0, 0, 255), -1)  # Red filled circle
    
    # Draw ball trajectory
    if len(ball_positions) > 1:
        # Create a copy of ball positions to avoid modifying during iteration
        positions = list(ball_positions)
        
        # Draw the trajectory lines
        for i in range(1, len(positions)):
            # Thicker line for more recent positions
            thickness = int(np.sqrt(len(positions) / float(i + 1)) * 2.5)
            cv2.line(frame, positions[i-1], positions[i], (0, 0, 255), thickness)
    
    # STEP 4: ADD TEAM COUNTS AT CENTER TOP (as requested)
    # Create text for team counts
    team_text = f"Left team: {left_players}  |  Right team: {right_players}"
    
    # Calculate text position to center it
    text_size = cv2.getTextSize(team_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
    text_x = (width - text_size[0]) // 2
    
    # Draw text with background for better visibility
    cv2.rectangle(frame, (text_x - 10, 5), (text_x + text_size[0] + 10, 35), (0, 0, 0), -1)
    cv2.putText(frame, team_text, (text_x, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Draw court dividing line
    cv2.line(frame, (mid_line, 0), (mid_line, height), (255, 255, 255), 2)
    
    # Write frame to output and display
    out.write(frame)
    
    # Optional: Check processing time to ensure we're not slowing down
    processing_time = time.time() - processing_start
    #frame_time = 1.0 / fps
    #if processing_time > frame_time:
    #    print(f"Warning: Processing too slow ({processing_time:.3f}s vs {frame_time:.3f}s needed)")
    
    # Display the resulting frame
    cv2.imshow('Olympic Volleyball Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. Output saved to 'output.avi'")


0: 384x640 18 persons, 320.6ms
Speed: 1.4ms preprocess, 320.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 372.4ms
Speed: 1.5ms preprocess, 372.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 323.9ms
Speed: 1.1ms preprocess, 323.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 320.8ms
Speed: 1.0ms preprocess, 320.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 312.5ms
Speed: 1.0ms preprocess, 312.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 305.2ms
Speed: 1.0ms preprocess, 305.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 298.6ms
Speed: 1.0ms preprocess, 298.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 298.4ms
Speed: 0.9ms preprocess, 298.4ms inference, 0.6ms postproc

In [2]:
import cv2
import numpy as np
from collections import deque
import time

# Initialize video capture
cap = cv2.VideoCapture('volleyball_match.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

# Ball tracking parameters
ball_positions = deque(maxlen=30)  # Store trajectory points
previous_frame = None  # For frame differencing
previous_ball_pos = None  # Last detected ball position

print("Starting volleyball ball tracking...")

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale for processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Initialize ball detection flag
    ball_detected = False
    
    # If we have a previous frame, use frame differencing to find motion
    if previous_frame is not None:
        # Frame difference to detect movement
        frame_diff = cv2.absdiff(previous_frame, gray)
        _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Variables for best ball candidate
        best_score = 0
        best_ball = None
        
        # Check each contour
        for contour in contours:
            # Small contours could be the ball (volleyball is small in Olympic videos)
            if 10 < cv2.contourArea(contour) < 400:  # Adjusted for small ball
                (x, y, w, h) = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                
                # Ball should be roughly circular
                if 0.7 < aspect_ratio < 1.3:
                    center_x = x + w // 2
                    center_y = y + h // 2
                    roi = frame[max(0, y):min(height, y+h), max(0, x):min(width, x+w)]
                    
                    if roi.size > 0:
                        # Check if this region is predominantly white (volleyball)
                        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                        # More lenient white range for Olympic lighting
                        lower_white = np.array([0, 0, 140])
                        upper_white = np.array([180, 60, 255])
                        white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                        white_ratio = cv2.countNonZero(white_mask) / (roi.shape[0] * roi.shape[1])
                        
                        # Calculate score based on whiteness and circularity
                        score = white_ratio * (1 - abs(1 - aspect_ratio))
                        
                        # If previous position is known, favor objects closer to last position
                        if previous_ball_pos:
                            distance = np.sqrt((center_x - previous_ball_pos[0])**2 + 
                                              (center_y - previous_ball_pos[1])**2)
                            # Lower score for objects too far from previous position
                            if distance > 100:  # Max expected movement between frames
                                score *= (100 / distance)
                        
                        # Update best candidate if this one is better
                        if score > best_score and white_ratio > 0.3:
                            best_score = score
                            best_ball = {
                                'center': (center_x, center_y),
                                'radius': max(w, h) // 2,
                                'score': score
                            }
        
        # If we found a good ball candidate
        if best_ball and best_score > 0.2:
            ball_x, ball_y = best_ball['center']
            ball_detected = True
            
            # Store current position
            previous_ball_pos = (ball_x, ball_y)
            
            # Add to trajectory
            ball_positions.append((ball_x, ball_y))
            
            # Draw current ball position - red circle
            cv2.circle(frame, (ball_x, ball_y), 10, (0, 0, 255), -1)
    
    # Update previous frame
    previous_frame = gray.copy()
    
    # Draw ball trajectory
    if len(ball_positions) > 1:
        for i in range(1, len(ball_positions)):
            # Thicker line for more recent positions
            thickness = int(np.sqrt(len(ball_positions) / float(i + 1)) * 2.5)
            cv2.line(frame, ball_positions[i-1], ball_positions[i], (0, 0, 255), thickness)
    
    # Write frame to output video
    out.write(frame)
    
    # Display the frame
    cv2.imshow('Volleyball Ball Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. Output saved to 'output.avi'")

Starting volleyball ball tracking...
Processing complete. Output saved to 'output.avi'


In [3]:
import cv2
import numpy as np
from collections import deque

# Initialize video capture
cap = cv2.VideoCapture('volleyball_match.mp4')
if not cap.isOpened():
    print("Error: Could not open video file")
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video info: {width}x{height}, {fps} FPS")

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

# Initialize variables for ball tracking
ball_positions = deque(maxlen=30)  # Store trajectory points
previous_frame = None
previous_ball_pos = None

# Parameters for ball detection
min_ball_area = 10
max_ball_area = 400
min_circularity = 0.7
max_circularity = 1.3
white_threshold = 0.3

print("Starting volleyball ball tracking...")

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale for processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # If we have a previous frame, use frame differencing to find motion
    if previous_frame is not None:
        # Detect motion using frame differencing
        frame_diff = cv2.absdiff(previous_frame, gray)
        _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        # Find contours of moving objects
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Variables for best ball candidate
        best_score = 0
        best_ball = None
        
        # Check each contour
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Filter by size - volleyball is small in frame
            if min_ball_area < area < max_ball_area:
                # Get bounding rectangle
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                
                # Check if shape is roughly circular
                if min_circularity < aspect_ratio < max_circularity:
                    center_x = x + w // 2
                    center_y = y + h // 2
                    
                    # Extract ROI for color analysis
                    roi = frame[max(0, y):min(height, y+h), max(0, x):min(width, x+w)]
                    
                    if roi.size > 0:
                        # Check if region is white (volleyball)
                        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                        lower_white = np.array([0, 0, 140])
                        upper_white = np.array([180, 60, 255])
                        white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                        white_ratio = cv2.countNonZero(white_mask) / (roi.shape[0] * roi.shape[1])
                        
                        # Score based on whiteness and circularity
                        circularity_factor = 1 - abs(1 - aspect_ratio)
                        score = white_ratio * circularity_factor
                        
                        # Consider previous position for tracking consistency
                        if previous_ball_pos:
                            distance = np.sqrt((center_x - previous_ball_pos[0])**2 + 
                                              (center_y - previous_ball_pos[1])**2)
                            # Lower score for objects too far from previous position
                            if distance > 100:  # Max expected movement between frames
                                score *= (100 / distance)
                        
                        # Update best candidate if this one is better
                        if score > best_score and white_ratio > white_threshold:
                            best_score = score
                            best_ball = {
                                'center': (center_x, center_y),
                                'radius': max(w, h) // 2,
                                'score': score
                            }
        
        # If we found a good ball candidate
        if best_ball and best_score > 0.2:
            ball_x, ball_y = best_ball['center']
            
            # Store current position
            previous_ball_pos = (ball_x, ball_y)
            
            # Add to trajectory
            ball_positions.append((ball_x, ball_y))
            
            # Draw current ball position - red circle
            cv2.circle(frame, (ball_x, ball_y), 10, (0, 0, 255), -1)
    
    # Update previous frame
    previous_frame = gray.copy()
    
    # Draw ball trajectory
    if len(ball_positions) > 1:
        for i in range(1, len(ball_positions)):
            # Thicker line for more recent positions
            thickness = int(np.sqrt(len(ball_positions) / float(i + 1)) * 2.5)
            cv2.line(frame, ball_positions[i-1], ball_positions[i], (0, 0, 255), thickness)
    
    # Write frame to output video
    out.write(frame)
    
    # Display the frame
    cv2.imshow('Volleyball Ball Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. Output saved to 'output.avi'")

Video info: 1280x720, 28 FPS
Starting volleyball ball tracking...


KeyboardInterrupt: 

In [4]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture('volleyball_match.mp4')
if not cap.isOpened():
    print("Error: Could not open video")
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))

# Initialize background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=120, varThreshold=40, detectShadows=False)

# Kalman Filter setup for smooth tracking
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03

# Last known position of the ball
last_ball_position = None
frames_without_detection = 0
MAX_FRAMES_TO_PREDICT = 5

print("Processing video...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Create a copy for drawing
    output_frame = frame.copy()
    
    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)
    
    # Apply thresholding to get clearer mask
    _, thresh = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)
    
    # Apply morphological operations to remove noise
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # Find contours of moving objects
    contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Variables for ball detection
    ball_detected = False
    best_ball = None
    best_score = 0
    
    # Check each contour
    for contour in contours:
        area = cv2.contourArea(contour)
        
        # Filter by size - volleyball size range
        if 50 < area < 900:  # Adjust these thresholds based on your video
            x, y, w, h = cv2.boundingRect(contour)
            
            # Check if shape is roughly circular
            aspect_ratio = float(w) / h
            if 0.7 < aspect_ratio < 1.3:
                
                # Extract the ROI
                roi = frame[y:y+h, x:x+w]
                
                if roi.size > 0:
                    # Check if it's predominantly white (volleyball)
                    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                    lower_white = np.array([0, 0, 150])
                    upper_white = np.array([180, 50, 255])
                    white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                    white_ratio = cv2.countNonZero(white_mask) / (roi.shape[0] * roi.shape[1])
                    
                    # Calculate score based on whiteness and circularity
                    circularity_score = 1 - abs(1 - aspect_ratio)
                    ball_score = white_ratio * circularity_score
                    
                    # If we have a previous position, consider proximity
                    if last_ball_position is not None:
                        center_x, center_y = x + w//2, y + h//2
                        distance = np.sqrt((center_x - last_ball_position[0])**2 + 
                                          (center_y - last_ball_position[1])**2)
                        
                        # Favor objects closer to last position, but not too strict
                        proximity_factor = max(0.3, min(1.0, 100 / max(distance, 1)))
                        ball_score *= proximity_factor
                    
                    # Update best candidate
                    if ball_score > best_score and white_ratio > 0.3:
                        best_score = ball_score
                        best_ball = {
                            'x': x, 'y': y, 'w': w, 'h': h,
                            'center': (x + w//2, y + h//2),
                            'radius': max(w, h) // 2
                        }
    
    # If we found a ball
    if best_ball and best_score > 0.2:
        ball_detected = True
        frames_without_detection = 0
        
        # Update Kalman filter
        kalman.correct(np.array([np.float32(best_ball['center'][0]), np.float32(best_ball['center'][1])]))
        
        # Save position
        last_ball_position = best_ball['center']
        
        # Draw circle around the ball - BRIGHT GREEN, THICK CIRCLE
        cv2.circle(output_frame, best_ball['center'], best_ball['radius'], (0, 255, 0), 3)
        
        # Add a highlight in the center
        cv2.circle(output_frame, best_ball['center'], 3, (0, 0, 255), -1)
        
    else:
        # Ball not detected in this frame
        frames_without_detection += 1
        
        # Use Kalman prediction if we recently had a detection
        if last_ball_position is not None and frames_without_detection < MAX_FRAMES_TO_PREDICT:
            prediction = kalman.predict()
            pred_x, pred_y = int(prediction[0, 0]), int(prediction[1, 0])
            
            # Only use prediction if it's within the frame
            if 0 <= pred_x < width and 0 <= pred_y < height:
                # Draw predicted position with different color - YELLOW, DASHED
                cv2.circle(output_frame, (pred_x, pred_y), 15, (0, 255, 255), 2)
                cv2.drawMarker(output_frame, (pred_x, pred_y), (0, 255, 255), cv2.MARKER_CROSS, 10, 2)
                
                # Update last position with prediction
                last_ball_position = (pred_x, pred_y)
    
    # Write to output
    out.write(output_frame)
    
    # Display the result
    cv2.imshow('Volleyball Tracking', output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. Output saved to 'output.avi'")

Processing video...
Processing complete. Output saved to 'output.avi'


In [5]:
import cv2
import numpy as np
import time
from collections import deque
import math
from ultralytics import YOLO

class VolleyballTracker:
    def __init__(self, video_path='volleyball_match.mp4', output_path='output.avi'):
        # Video properties
        self.cap = cv2.VideoCapture(video_path)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Output video setup
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.out = cv2.VideoWriter(output_path, fourcc, self.fps, (self.width, self.height))
        
        # Ball tracking variables
        self.ball_history = deque(maxlen=30)  # Store recent confirmed positions
        self.candidate_buffer = []  # Store candidate positions for voting
        self.last_reliable_position = None
        self.frames_since_detection = 0
        self.MAX_FRAMES_WITHOUT_DETECTION = 10
        
        # Initialize tracking algorithms
        self.setup_trackers()
        
        # Pre-load first frame for initialization
        ret, first_frame = self.cap.read()
        if not ret:
            raise Exception("Could not read video file")
        self.prev_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
        
        # Debug variables
        self.frame_count = 0
        self.detection_count = 0
        self.detection_methods = {'yolo': 0, 'hough': 0, 'motion': 0, 'color': 0, 'optical': 0}
        
        print(f"Tracker initialized for video: {self.width}x{self.height} @ {self.fps}fps")

    def setup_trackers(self):
        # 1. YOLO model for ball detection
        try:
            self.yolo_model = YOLO('yolov8x.pt')  # Use largest model for best accuracy
            print("YOLO model loaded successfully")
        except Exception as e:
            print(f"Warning: YOLO initialization failed: {e}")
            self.yolo_model = None
        
        # 2. Background subtractor for motion detection
        self.bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=120, varThreshold=40, detectShadows=False)
        
        # 3. Kalman filter for trajectory prediction
        self.kalman = cv2.KalmanFilter(6, 3)  # [x, y, z, dx, dy, dz] and [x, y, z]
        self.kalman.measurementMatrix = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0]], np.float32)
        self.kalman.transitionMatrix = np.array([
            [1, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 0],
            [0, 0, 1, 0, 0, 1],
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1]], np.float32)
        self.kalman.processNoiseCov = np.eye(6, dtype=np.float32) * 0.03
        self.kalman_initialized = False
        
        # 4. Optical flow for tracking movement
        self.lk_params = dict(winSize=(15, 15),
                         maxLevel=2,
                         criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        self.feature_params = dict(maxCorners=100,
                              qualityLevel=0.3,
                              minDistance=7,
                              blockSize=7)
        self.optical_points = []

    def detect_ball_yolo(self, frame):
        """Detect ball using YOLO model"""
        if self.yolo_model is None:
            return []
        
        results = self.yolo_model(frame, conf=0.3, classes=32)  # Class 32 is sports ball in COCO
        candidates = []
        
        for r in results:
            boxes = r.boxes
            for box in boxes:
                conf = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                radius = max((x2 - x1), (y2 - y1)) // 2
                
                # Extract ROI to check whiteness
                ball_roi = frame[max(0, y1):min(self.height, y2), max(0, x1):min(self.width, x2)]
                if ball_roi.size > 0:
                    # Check if predominantly white (volleyball)
                    hsv_roi = cv2.cvtColor(ball_roi, cv2.COLOR_BGR2HSV)
                    lower_white = np.array([0, 0, 150])
                    upper_white = np.array([180, 60, 255])
                    white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                    white_ratio = cv2.countNonZero(white_mask) / (ball_roi.shape[0] * ball_roi.shape[1])
                    
                    if white_ratio > 0.3 or conf > 0.5:  # Accept if very white or high confidence
                        candidates.append({
                            'center': (center_x, center_y),
                            'radius': radius,
                            'score': conf * (1 + white_ratio),  # Higher score for whiter objects
                            'method': 'yolo'
                        })
        
        return candidates

    def detect_ball_hough(self, frame):
        """Detect circular objects using Hough Circle Transform"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)
        
        # Detect circles with different parameter combinations for robustness
        circles = None
        param_combinations = [
            # (dp, minDist, param1, param2, minRadius, maxRadius)
            (1.2, 20, 50, 30, 10, 30),
            (1.5, 30, 70, 25, 5, 40),
            (1.0, 25, 60, 20, 8, 35)
        ]
        
        for dp, min_dist, p1, p2, min_r, max_r in param_combinations:
            detected_circles = cv2.HoughCircles(
                blurred, cv2.HOUGH_GRADIENT, dp=dp, minDist=min_dist,
                param1=p1, param2=p2, minRadius=min_r, maxRadius=max_r
            )
            
            if detected_circles is not None:
                circles = detected_circles
                break
        
        candidates = []
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0, :]:
                x, y, r = circle
                
                # Skip if out of frame bounds
                if x < 0 or y < 0 or x >= self.width or y >= self.height:
                    continue
                    
                # Extract ROI
                roi_x1 = max(0, x - r)
                roi_y1 = max(0, y - r)
                roi_x2 = min(self.width, x + r)
                roi_y2 = min(self.height, y + r)
                
                roi = frame[roi_y1:roi_y2, roi_x1:roi_x2]
                if roi.size == 0:
                    continue
                    
                # Check whiteness
                hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                lower_white = np.array([0, 0, 150])
                upper_white = np.array([180, 60, 255])
                white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                white_ratio = cv2.countNonZero(white_mask) / (roi.shape[0] * roi.shape[1])
                
                if white_ratio > 0.25:  # Accept if sufficiently white
                    candidates.append({
                        'center': (x, y),
                        'radius': r,
                        'score': white_ratio,
                        'method': 'hough'
                    })
        
        return candidates

    def detect_ball_motion(self, frame, prev_frame):
        """Detect ball using motion difference and color filtering"""
        # Convert frames to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate absolute difference
        frame_diff = cv2.absdiff(prev_frame, gray)
        _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        # Apply foreground mask from background subtraction
        fg_mask = self.bg_subtractor.apply(frame)
        _, fg_thresh = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)
        
        # Combine both masks
        combined_mask = cv2.bitwise_and(thresh, fg_thresh)
        
        # Find contours
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        candidates = []
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Filter by size
            if 50 < area < 800:  # Adjusted for volleyball size
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                
                # Check if roughly circular
                if 0.7 < aspect_ratio < 1.3:
                    # Extract ROI
                    roi = frame[y:y+h, x:x+w]
                    
                    if roi.size > 0:
                        # Check if predominantly white
                        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                        lower_white = np.array([0, 0, 150])
                        upper_white = np.array([180, 60, 255])
                        white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                        white_ratio = cv2.countNonZero(white_mask) / (roi.shape[0] * roi.shape[1])
                        
                        circularity = 4 * np.pi * area / (cv2.arcLength(contour, True) ** 2) if cv2.arcLength(contour, True) > 0 else 0
                        
                        if white_ratio > 0.25:
                            score = white_ratio * circularity
                            candidates.append({
                                'center': (x + w//2, y + h//2),
                                'radius': max(w, h) // 2,
                                'score': score,
                                'method': 'motion'
                            })
        
        return candidates

    def track_ball_optical_flow(self, prev_frame, current_frame, prev_points):
        """Track ball using optical flow"""
        if len(prev_points) == 0:
            return []
        
        # Convert to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) if len(prev_frame.shape) > 2 else prev_frame
        current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY) if len(current_frame.shape) > 2 else current_frame
        
        # Calculate optical flow
        new_points, status, error = cv2.calcOpticalFlowPyrLK(
            prev_gray, current_gray, 
            np.array(prev_points, dtype=np.float32), 
            None, 
            **self.lk_params
        )
        
        candidates = []
        if new_points is not None:
            # Filter only good points
            good_new = new_points[status == 1]
            
            for point in good_new:
                x, y = point.ravel()
                x, y = int(x), int(y)
                
                # Check if point is within frame bounds
                if 0 <= x < self.width and 0 <= y < self.height:
                    # Extract small region around point
                    radius = 20  # Extract a bit larger area around flow point
                    x1 = max(0, x - radius)
                    y1 = max(0, y - radius)
                    x2 = min(self.width, x + radius)
                    y2 = min(self.height, y + radius)
                    
                    roi = current_frame[y1:y2, x1:x2]
                    if roi.size > 0:
                        # Check whiteness
                        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                        lower_white = np.array([0, 0, 150])
                        upper_white = np.array([180, 60, 255])
                        white_mask = cv2.inRange(hsv_roi, lower_white, upper_white)
                        white_ratio = cv2.countNonZero(white_mask) / (roi.shape[0] * roi.shape[1])
                        
                        if white_ratio > 0.2:
                            candidates.append({
                                'center': (x, y),
                                'radius': 15,  # Approximate radius
                                'score': white_ratio,
                                'method': 'optical'
                            })
        
        return candidates

    def detect_ball_color(self, frame):
        """Detect ball using direct color filtering for white volleyball"""
        # Convert to HSV and create white mask
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Volleyball is typically white
        lower_white = np.array([0, 0, 190])
        upper_white = np.array([180, 30, 255])
        white_mask = cv2.inRange(hsv, lower_white, upper_white)
        
        # Apply morphological operations to clean up the mask
        kernel = np.ones((5, 5), np.uint8)
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel)
        white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel)
        
        # Find contours of white regions
        contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        candidates = []
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Filter by size (volleyball size in the frame)
            if 50 < area < 1000:
                # Calculate circularity
                perimeter = cv2.arcLength(contour, True)
                if perimeter == 0:
                    continue
                circularity = 4 * np.pi * area / (perimeter ** 2)
                
                # If somewhat circular
                if circularity > 0.5:
                    x, y, w, h = cv2.boundingRect(contour)
                    center_x, center_y = x + w//2, y + h//2
                    radius = max(w, h) // 2
                    
                    # Score based on circularity and size
                    score = circularity * min(1.0, (area / 400))  # Normalize to prefer ~400px area
                    
                    candidates.append({
                        'center': (center_x, center_y),
                        'radius': radius,
                        'score': score,
                        'method': 'color'
                    })
        
        return candidates

    def update_kalman(self, measurement, measurement_noise=0.1):
        """Update Kalman filter with new measurement"""
        if not self.kalman_initialized:
            # Initialize Kalman state
            self.kalman.statePre = np.zeros((6, 1), np.float32)
            self.kalman.statePre[0] = measurement[0]
            self.kalman.statePre[1] = measurement[1]
            self.kalman.statePre[2] = 0  # Z position (depth)
            
            self.kalman.statePost = self.kalman.statePre.copy()
            
            # Initialize measurement noise
            self.kalman.measurementNoiseCov = np.eye(3, dtype=np.float32) * measurement_noise
            
            self.kalman_initialized = True
            return measurement
        
        # Predict next position
        prediction = self.kalman.predict()
        
        # Correct with measurement
        measurement_array = np.array([[np.float32(measurement[0])], 
                                     [np.float32(measurement[1])],
                                     [np.float32(0)]])  # Z=0 assumed for 2D tracking
        
        corrected = self.kalman.correct(measurement_array)
        
        return (int(corrected[0, 0]), int(corrected[1, 0]))

    def vote_for_ball(self, candidates):
        """Use consensus voting to determine most likely ball position"""
        if not candidates:
            return None
            
        # If only one candidate, return it
        if len(candidates) == 1:
            return candidates[0]
            
        # Score candidates based on multiple factors
        for candidate in candidates:
            final_score = candidate['score']
            
            # Factor 1: Distance to last reliable position
            if self.last_reliable_position:
                last_x, last_y = self.last_reliable_position
                curr_x, curr_y = candidate['center']
                distance = math.sqrt((last_x - curr_x)**2 + (last_y - curr_y)**2)
                
                # Penalize candidates too far from last position (but not too much)
                max_distance = 100  # Maximum reasonable distance between frames
                distance_factor = max(0.3, min(1.0, 1 - (distance / max_distance)))
                final_score *= distance_factor
            
            # Factor 2: Reward consistent detection methods
            if len(self.ball_history) > 0 and self.detection_methods[candidate['method']] > 0:
                method_confidence = self.detection_methods[candidate['method']] / max(self.detection_methods.values())
                final_score *= (0.7 + 0.3 * method_confidence)  # Scale factor to not over-penalize
            
            # Factor 3: Consider detection method reliability
            # YOLO and Hough are generally more reliable for initial detection
            method_reliability = {
                'yolo': 1.2,
                'hough': 1.1,
                'motion': 1.0,
                'optical': 0.9,
                'color': 0.8
            }
            final_score *= method_reliability[candidate['method']]
            
            candidate['final_score'] = final_score
            
        # Sort by final score and return best candidate
        candidates.sort(key=lambda x: x['final_score'], reverse=True)
        best_candidate = candidates[0]
        
        # Update detection method counter
        self.detection_methods[best_candidate['method']] += 1
        
        return best_candidate

    def process_video(self):
        """Process video frame by frame and track the volleyball"""
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    break
                    
                self.frame_count += 1
                
                # Create a copy for drawing
                output_frame = frame.copy()
                
                # Extract grayscale for processing
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                # Collect candidates from all detection methods
                candidates = []
                
                # Method 1: YOLO detection
                yolo_candidates = self.detect_ball_yolo(frame)
                candidates.extend(yolo_candidates)
                
                # Method 2: Hough Circle Transform
                hough_candidates = self.detect_ball_hough(frame)
                candidates.extend(hough_candidates)
                
                # Method 3: Motion-based detection
                motion_candidates = self.detect_ball_motion(frame, self.prev_frame)
                candidates.extend(motion_candidates)
                
                # Method 4: Color-based detection 
                color_candidates = self.detect_ball_color(frame)
                candidates.extend(color_candidates)
                
                # Method 5: Optical flow tracking
                if self.optical_points:
                    optical_candidates = self.track_ball_optical_flow(self.prev_frame, frame, self.optical_points)
                    candidates.extend(optical_candidates)
                
                # Update previous frame
                self.prev_frame = gray
                
                # Get the best ball candidate through voting
                best_candidate = self.vote_for_ball(candidates)
                
                if best_candidate:
                    # Ball detected
                    self.frames_since_detection = 0
                    self.detection_count += 1
                    
                    # Apply Kalman filter for smoothing
                    smoothed_position = self.update_kalman(best_candidate['center'])
                    
                    # Update tracking variables
                    self.last_reliable_position = smoothed_position
                    self.ball_history.append(smoothed_position)
                    
                    # Update optical flow points
                    self.optical_points = [np.array([smoothed_position], dtype=np.float32)]
                    
                    # Draw ball boundary - BRIGHT GREEN, THICK CIRCLE
                    cv2.circle(output_frame, smoothed_position, best_candidate['radius'], (0, 255, 0), 3)
                    
                    # Add a marker at center
                    cv2.drawMarker(output_frame, smoothed_position, (0, 0, 255), cv2.MARKER_CROSS, 10, 2)
                    
                    # Display detection method
                    cv2.putText(output_frame, f"Method: {best_candidate['method']}", 
                              (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                else:
                    # Ball not detected in this frame
                    self.frames_since_detection += 1
                    
                    if self.last_reliable_position and self.frames_since_detection < self.MAX_FRAMES_WITHOUT_DETECTION:
                        # Predict position using Kalman filter
                        prediction = self.kalman.predict()
                        pred_x, pred_y = int(prediction[0, 0]), int(prediction[1, 0])
                        
                        # Only use prediction if it's within the frame
                        if 0 <= pred_x < self.width and 0 <= pred_y < self.height:
                            # Draw predicted position - YELLOW, DASHED
                            cv2.circle(output_frame, (pred_x, pred_y), 15, (0, 255, 255), 2)
                            cv2.drawMarker(output_frame, (pred_x, pred_y), (0, 255, 255), cv2.MARKER_CROSS, 10, 2)
                            
                            # Update optical flow points with prediction
                            self.optical_points = [np.array([[pred_x, pred_y]], dtype=np.float32)]
                
                # Add frame number and detection stats
                cv2.putText(output_frame, f"Frame: {self.frame_count}/{self.total_frames}", 
                          (self.width - 200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Write to output
                self.out.write(output_frame)
                
                # Display the result (optional)
                cv2.imshow('Volleyball Tracking', output_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
                # Print progress periodically
                if self.frame_count % 100 == 0:
                    detection_rate = (self.detection_count / self.frame_count) * 100
                    print(f"Processed {self.frame_count}/{self.total_frames} frames ({detection_rate:.1f}% detection rate)")
                    print(f"Detection methods: {self.detection_methods}")
                    
        except Exception as e:
            print(f"Error during processing: {e}")
        finally:
            # Release resources
            self.cap.release()
            self.out.release()
            cv2.destroyAllWindows()
            
            # Print final stats
            if self.frame_count > 0:
                detection_rate = (self.detection_count / self.frame_count) * 100
                print(f"\nProcessing complete. Output saved to 'output.avi'")
                print(f"Processed {self.frame_count} frames with {self.detection_count} ball detections")
                print(f"Overall detection rate: {detection_rate:.1f}%")
                print(f"Detection methods used: {self.detection_methods}")

# Run the tracker
if __name__ == "__main__":
    start_time = time.time()
    tracker = VolleyballTracker()
    tracker.process_video()
    elapsed = time.time() - start_time
    print(f"Total processing time: {elapsed:.2f} seconds")

YOLO model loaded successfully
Tracker initialized for video: 1280x720 @ 28fps

0: 384x640 (no detections), 357.1ms
Speed: 5.2ms preprocess, 357.1ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)



/var/folders/r0/kmkzmgh52q9f32w030w6nl_h0000gn/T/ipykernel_7212/788875630.py:155: RuntimeWarning: overflow encountered in scalar subtract
  roi_y1 = max(0, y - r)


0: 384x640 (no detections), 391.7ms
Speed: 1.8ms preprocess, 391.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)



/var/folders/r0/kmkzmgh52q9f32w030w6nl_h0000gn/T/ipykernel_7212/788875630.py:383: RuntimeWarning: overflow encountered in scalar subtract
  distance = math.sqrt((last_x - curr_x)**2 + (last_y - curr_y)**2)
/var/folders/r0/kmkzmgh52q9f32w030w6nl_h0000gn/T/ipykernel_7212/788875630.py:383: RuntimeWarning: overflow encountered in scalar add
  distance = math.sqrt((last_x - curr_x)**2 + (last_y - curr_y)**2)


0: 384x640 (no detections), 382.0ms
Speed: 1.2ms preprocess, 382.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 351.2ms
Speed: 1.4ms preprocess, 351.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 320.1ms
Speed: 1.0ms preprocess, 320.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 sports ball, 306.9ms
Speed: 1.0ms preprocess, 306.9ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 sports ball, 317.6ms
Speed: 0.9ms preprocess, 317.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 304.8ms
Speed: 0.9ms preprocess, 304.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 313.7ms
Speed: 1.0ms preprocess, 313.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 303.5ms
Speed: 1.0ms preprocess

In [1]:
import cv2
import numpy as np
import os
import gdown
from collections import deque

def download_files():
    """Downloads the video and YOLO model files if they don't exist."""
    files = {
        # Video file from Google Drive
        "volleyball.mp4": "1JLNjXLnRJUWbzbH1V11DjzPrgjYswGDJ",
        # YOLOv3 model files
        "yolov3.weights": "https://pjreddie.com/media/files/yolov3.weights",
        "yolov3.cfg": "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg",
        "coco.names": "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"
    }
    
    for filename, url_or_id in files.items():
        if not os.path.exists(filename):
            print(f"Downloading {filename}...")
            if "drive.google.com" in url_or_id or len(url_or_id) < 40: # Heuristic for GDrive ID
                 gdown.download(id=url_or_id, output=filename, quiet=False)
            else:
                 gdown.download(url=url_or_id, output=filename, quiet=False)
        else:
            print(f"{filename} already exists.")

def load_yolo_model():
    """Loads the YOLOv3 model and class names."""
    print("\nLoading YOLO model...")
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    output_layers = [net.getLayerNames()[i - 1] for i in net.getUnconnectedOutLayers()]
    print("YOLO model loaded successfully.")
    return net, classes, output_layers

def count_players(video_path, net, classes, output_layers):
    """Counts players on each team based on their position in a single frame."""
    print("\n--- Starting Player Count Analysis ---")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video for player counting.")
        return

    # Read a specific frame (e.g., frame 150) where players are visible
    cap.set(cv2.CAP_PROP_POS_FRAMES, 150)
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read a suitable frame for counting.")
        cap.release()
        return

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Use Non-Max Suppression to remove duplicate boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    team1_count = 0
    team2_count = 0
    center_line_x = width // 2

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            # Simple assumption: team is decided by which side of the court they are on
            if (x + w/2) < center_line_x:
                team1_count += 1
            else:
                team2_count += 1

    print("\n--- Player Count Results ---")
    print(f"Team 1 (left side) players found: {team1_count}")
    print(f"Team 2 (right side) players found: {team2_count}")
    print("Note: This count is based on a single frame and positional analysis.")
    
    cap.release()


def track_ball_trajectory(video_path, output_path, net, classes, output_layers):
    """Tracks the ball, highlights it, and draws its trajectory."""
    print("\n--- Starting Ball Tracking and Trajectory Visualization ---")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video for tracking.")
        return

    # Get video properties for the output writer
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    # Deque to store the ball's center points for the trail
    trajectory_points = deque(maxlen=32)

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        print(f"Processing frame {frame_count}/{total_frames}...", end='\r')

        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        detections = net.forward(output_layers)

        ball_box = None
        max_confidence = 0.0

        for detection_layer in detections:
            for detection in detection_layer:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                # We are looking for 'sports ball'
                if confidence > 0.4 and classes[class_id] == "sports ball":
                    # A frame might have multiple "ball" detections; pick the most confident one
                    if confidence > max_confidence:
                        max_confidence = confidence
                        center_x = int(detection[0] * frame_width)
                        center_y = int(detection[1] * frame_height)
                        w = int(detection[2] * frame_width)
                        h = int(detection[3] * frame_height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        ball_box = (x, y, w, h, center_x, center_y)

        # If a ball was detected in this frame
        if ball_box is not None:
            x, y, w, h, cx, cy = ball_box
            # Highlight the ball's current location
            cv2.circle(frame, (cx, cy), 15, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Add the center point to our trajectory list
            trajectory_points.appendleft((cx, cy))
        
        # Draw the trajectory trail
        for i in range(1, len(trajectory_points)):
            if trajectory_points[i - 1] is None or trajectory_points[i] is None:
                continue
            # Make the trail thicker at the beginning and thinner at the end
            thickness = int(np.sqrt(len(trajectory_points) / float(i + 1)) * 2.5)
            cv2.line(frame, trajectory_points[i - 1], trajectory_points[i], (0, 0, 255), thickness)

        # Write the processed frame to the output video
        out.write(frame)
        
        # Optional: display the frame while processing
        # cv2.imshow("Volleyball Tracking", frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break

    print("\nProcessing complete.")
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Output video saved to: {output_path}")

if __name__ == "__main__":
    # 1. Download necessary files
    download_files()

    # 2. Load the deep learning model
    net, classes, output_layers = load_yolo_model()

    video_file = "volleyball.mp4"
    output_video_file = "volleyball_tracked.mp4"

    # 3. Perform player count analysis
    count_players(video_file, net, classes, output_layers)

    # 4. Perform ball tracking and create the output video
    track_ball_trajectory(video_file, output_video_file, net, classes, output_layers)

Downloading...
From: https://drive.google.com/uc?id=1JLNjXLnRJUWbzbH1V11DjzPrgjYswGDJ
To: /Users/mithil/DVA/LAB9/volleyball.mp4
100%|██████████████████████████████████████| 30.7M/30.7M [00:01<00:00, 16.5MB/s]


Downloading...
From: https://pjreddie.com/media/files/yolov3.weights
To: /Users/mithil/DVA/LAB9/yolov3.weights
9.09kB [00:00, 2.68MB/s]


Downloading...
From: https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
To: /Users/mithil/DVA/LAB9/yolov3.cfg
8.34kB [00:00, 3.75MB/s]                                                        


Downloading...
From: https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
To: /Users/mithil/DVA/LAB9/coco.names
625B [00:00, 557kB/s]                                                           


Loading YOLO model...


error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/dnn/src/darknet/darknet_io.cpp:991: error: (-213:The function/feature is not implemented) Transpose the weights (except for convolutional) is not implemented in function 'ReadDarknetFromWeightsStream'


In [9]:
!pip install opencv-python numpy gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gdown]


In [2]:
import cv2
import numpy as np
from collections import deque
import os
import gdown

def download_video():
    """Downloads the volleyball video if it doesn't exist."""
    video_file = "volleyball.mp4"
    video_id = "1JLNjXLnRJUWbzbH1V11DjzPrgjYswGDJ"
    
    if not os.path.exists(video_file):
        print(f"Downloading {video_file}...")
        gdown.download(id=video_id, output=video_file, quiet=False)
    else:
        print(f"{video_file} already exists.")
    return video_file

class VolleyballTracker:
    def __init__(self, video_path, output_path="output.avi"):
        """Initialize the volleyball tracker."""
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise ValueError(f"Could not open video: {video_path}")
            
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        
        # Create output video writer
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.out = cv2.VideoWriter(output_path, fourcc, self.fps, (self.width, self.height))
        
        # Initialize tracking variables
        self.ball_positions = deque(maxlen=30)
        self.last_position = None
        self.bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=120, varThreshold=36)
        self.prev_frame = None
        
        # Kalman filter for smoother tracking
        self.kalman = cv2.KalmanFilter(4, 2)  # 4 state variables, 2 measurement variables
        self.kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
        self.kalman.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03
        self.kalman_initialized = False
    
    def detect_ball(self, frame):
        """Detect volleyball using multiple methods."""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # 1. Apply background subtraction
        fg_mask = self.bg_subtractor.apply(frame)
        _, thresh_bg = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)
        
        # 2. Use frame differencing if we have a previous frame
        if self.prev_frame is not None:
            frame_diff = cv2.absdiff(self.prev_frame, gray)
            _, thresh_diff = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
            thresh_diff = cv2.dilate(thresh_diff, None, iterations=2)
            motion_mask = thresh_diff
        else:
            motion_mask = np.zeros_like(gray)
        
        # 3. Use color detection for white volleyball
        lower_white = np.array([0, 0, 180])
        upper_white = np.array([180, 55, 255])
        white_mask = cv2.inRange(hsv, lower_white, upper_white)
        
        # Combine masks
        combined_mask = cv2.bitwise_and(thresh_bg, white_mask)
        if self.prev_frame is not None:
            combined_mask = cv2.bitwise_or(combined_mask, cv2.bitwise_and(motion_mask, white_mask))
        
        # Clean up with morphological operations
        kernel = np.ones((5, 5), np.uint8)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel, iterations=1)
        
        # Find contours
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Update previous frame
        self.prev_frame = gray
        
        # Initialize variables to find best ball candidate
        best_ball = None
        best_score = 0
        
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Filter by size (volleyball size range)
            if 50 < area < 800:
                # Calculate circularity
                perimeter = cv2.arcLength(contour, True)
                if perimeter == 0:
                    continue
                circularity = 4 * np.pi * area / (perimeter ** 2)
                
                # Check if it's circular enough
                if circularity > 0.7:
                    x, y, w, h = cv2.boundingRect(contour)
                    aspect_ratio = float(w) / h
                    
                    # Check if it's roughly circular by aspect ratio
                    if 0.7 < aspect_ratio < 1.3:
                        center_x = x + w//2
                        center_y = y + h//2
                        
                        # Calculate score based on multiple factors
                        score = circularity * min(1.0, aspect_ratio if aspect_ratio <= 1 else 1/aspect_ratio)
                        
                        # Check whiteness
                        roi = frame[y:y+h, x:x+w]
                        if roi.size > 0:
                            hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                            white_mask_roi = cv2.inRange(hsv_roi, lower_white, upper_white)
                            white_ratio = cv2.countNonZero(white_mask_roi) / (roi.shape[0] * roi.shape[1])
                            score *= white_ratio
                        
                        # Factor in proximity to last detection
                        if self.last_position is not None:
                            dist = np.sqrt((center_x - self.last_position[0])**2 + (center_y - self.last_position[1])**2)
                            # If it's reasonably close to last position, increase score
                            proximity_factor = max(0.5, min(1.0, 100 / max(dist, 1)))
                            score *= proximity_factor
                        
                        # Update best candidate if better score
                        if score > best_score and score > 0.4:
                            best_score = score
                            best_ball = {
                                'center': (center_x, center_y),
                                'radius': max(w, h) // 2,
                                'score': score,
                                'box': (x, y, w, h)
                            }
        
        return best_ball
    
    def update_kalman(self, measurement):
        """Update Kalman filter with new measurement."""
        if not self.kalman_initialized:
            # Initialize Kalman state
            self.kalman.statePre = np.array([[measurement[0]], [measurement[1]], [0], [0]], np.float32)
            self.kalman.statePost = self.kalman.statePre.copy()
            self.kalman_initialized = True
            return measurement
        
        # Predict next position
        prediction = self.kalman.predict()
        
        # Correct with measurement
        measurement_array = np.array([[np.float32(measurement[0])], [np.float32(measurement[1])]])
        corrected = self.kalman.correct(measurement_array)
        
        return (int(corrected[0, 0]), int(corrected[1, 0]))
    
    def track_ball(self):
        """Process video frame by frame to track the volleyball."""
        frame_count = 0
        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print("Starting volleyball tracking...")
        
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            
            frame_count += 1
            if frame_count % 30 == 0:
                print(f"Processing frame {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")
            
            # Detect ball in current frame
            ball = self.detect_ball(frame)
            
            if ball:
                # Apply Kalman filter for smoother tracking
                smooth_center = self.update_kalman(ball['center'])
                
                # Update tracking info
                self.last_position = smooth_center
                self.ball_positions.append(smooth_center)
                
                # Draw ball detection - BRIGHT GREEN CIRCLE
                cv2.circle(frame, smooth_center, ball['radius'], (0, 255, 0), 3)
                cv2.circle(frame, smooth_center, 3, (0, 0, 255), -1)  # Center point
            elif self.last_position is not None:
                # Use Kalman prediction if ball not detected
                prediction = self.kalman.predict()
                pred_x, pred_y = int(prediction[0, 0]), int(prediction[1, 0])
                
                # Check if prediction is within frame
                if 0 <= pred_x < self.width and 0 <= pred_y < self.height:
                    # Draw predicted position with yellow dashed circle
                    cv2.circle(frame, (pred_x, pred_y), 15, (0, 255, 255), 2)
            
            # Draw ball trajectory
            if len(self.ball_positions) > 1:
                for i in range(1, len(self.ball_positions)):
                    # Thicker line for more recent positions
                    thickness = int(np.sqrt(len(self.ball_positions) / float(i + 1)) * 2.5)
                    cv2.line(frame, self.ball_positions[i-1], self.ball_positions[i], (0, 0, 255), thickness)
            
            # Write frame to output video
            self.out.write(frame)
            
            # Optional: display frame (commented out for speed)
            # cv2.imshow('Volleyball Tracking', frame)
            # if cv2.waitKey(1) & 0xFF == ord('q'):
            #    break
        
        # Release resources
        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()
        print(f"Processing complete. Output saved to {self.out.name}")

if __name__ == "__main__":
    try:
        # Download video if needed
        video_file = download_video()
        
        # Create and run tracker
        tracker = VolleyballTracker(video_file, "volleyball_tracked.avi")
        tracker.track_ball()
    except Exception as e:
        print(f"Error: {e}")

volleyball.mp4 already exists.
Starting volleyball tracking...
Processing frame 30/3437 (0.9%)
Processing frame 60/3437 (1.7%)
Processing frame 90/3437 (2.6%)
Processing frame 120/3437 (3.5%)
Processing frame 150/3437 (4.4%)
Processing frame 180/3437 (5.2%)
Processing frame 210/3437 (6.1%)
Processing frame 240/3437 (7.0%)
Processing frame 270/3437 (7.9%)
Processing frame 300/3437 (8.7%)
Processing frame 330/3437 (9.6%)
Processing frame 360/3437 (10.5%)
Processing frame 390/3437 (11.3%)
Processing frame 420/3437 (12.2%)
Processing frame 450/3437 (13.1%)
Processing frame 480/3437 (14.0%)
Processing frame 510/3437 (14.8%)
Processing frame 540/3437 (15.7%)
Processing frame 570/3437 (16.6%)
Processing frame 600/3437 (17.5%)
Processing frame 630/3437 (18.3%)
Processing frame 660/3437 (19.2%)
Processing frame 690/3437 (20.1%)
Processing frame 720/3437 (20.9%)
Processing frame 750/3437 (21.8%)
Processing frame 780/3437 (22.7%)
Processing frame 810/3437 (23.6%)
Processing frame 840/3437 (24.4%)

In [3]:
import cv2
import numpy as np
import os
import gdown
import requests
from collections import deque

def download_file_from_url(url, filename):
    """A robust function to download a file from a URL with progress."""
    print(f"Downloading {filename} from {url}...")
    try:
        with requests.get(url, stream=True, timeout=30) as r:
            r.raise_for_status()  # Raise an exception for bad status codes
            total_size = int(r.headers.get('content-length', 0))
            block_size = 8192  # 8KB
            
            with open(filename, 'wb') as f:
                # Use a simple progress indicator
                downloaded_size = 0
                for chunk in r.iter_content(chunk_size=block_size):
                    f.write(chunk)
                    downloaded_size += len(chunk)
                    if total_size > 0:
                        progress = downloaded_size / total_size * 100
                        print(f"\r  -> Progress: {progress:.1f}% ({downloaded_size/1024/1024:.1f}MB / {total_size/1024/1024:.1f}MB)", end="")
        print(f"\nSuccessfully downloaded {filename}")
    except requests.exceptions.RequestException as e:
        print(f"\nError downloading {filename}: {e}")
        # Clean up partially downloaded file if it exists
        if os.path.exists(filename):
            os.remove(filename)

def download_files():
    """Downloads the video and YOLO model files if they don't exist."""
    files_to_download = {
        # Video file from Google Drive
        "volleyball.mp4": ("1JLNjXLnRJUWbzbH1V11DjzPrgjYswGDJ", "gdrive"),
        # YOLOv3 model files
        "yolov3.weights": ("https://pjreddie.com/media/files/yolov3.weights", "url"),
        "yolov3.cfg": ("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", "url"),
        "coco.names": ("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", "url")
    }

    for filename, (source, dl_type) in files_to_download.items():
        # Check if file exists and is valid
        file_is_valid = False
        if os.path.exists(filename):
            # For the large weights file, check if its size is reasonable (>200MB)
            if filename == "yolov3.weights":
                if os.path.getsize(filename) > 200 * 1024 * 1024:
                    print(f"{filename} already exists and seems valid.")
                    file_is_valid = True
                else:
                    print(f"{filename} exists but is too small. Deleting and redownloading.")
                    os.remove(filename)
            else:
                 print(f"{filename} already exists.")
                 file_is_valid = True

        # If the file doesn't exist or was invalid, download it
        if not file_is_valid:
            if dl_type == "gdrive":
                print(f"Downloading {filename} from Google Drive...")
                gdown.download(id=source, output=filename, quiet=False)
            else: # 'url'
                download_file_from_url(source, filename)


def load_yolo_model():
    """Loads the YOLOv3 model and class names."""
    print("\nLoading YOLO model...")
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    output_layers = [net.getLayerNames()[i - 1] for i in net.getUnconnectedOutLayers()]
    print("YOLO model loaded successfully.")
    return net, classes, output_layers

def count_players(video_path, net, classes, output_layers):
    """Counts players on each team based on their position in a single frame."""
    print("\n--- Starting Player Count Analysis ---")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video for player counting.")
        return

    # Read a specific frame (e.g., frame 150) where players are visible
    cap.set(cv2.CAP_PROP_POS_FRAMES, 150)
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read a suitable frame for counting.")
        cap.release()
        return

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes = []
    confidences = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":
                center_x = int(detection[0] * width)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(detection[1] * height - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    team1_count = 0
    team2_count = 0
    center_line_x = width // 2

    if hasattr(indexes, 'flatten'):
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            if (x + w/2) < center_line_x:
                team1_count += 1
            else:
                team2_count += 1

    print("\n--- Player Count Results ---")
    print(f"Team 1 (left side) players found: {team1_count}")
    print(f"Team 2 (right side) players found: {team2_count}")
    
    cap.release()


def track_ball_trajectory(video_path, output_path, net, classes, output_layers):
    """Tracks the ball, highlights it, and draws its trajectory."""
    print("\n--- Starting Ball Tracking and Trajectory Visualization ---")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video for tracking.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    trajectory_points = deque(maxlen=32)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        print(f"Processing frame {frame_count + 1}/{total_frames}...", end='\r')

        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        detections = net.forward(output_layers)

        ball_box = None
        max_confidence = 0.0

        for detection_layer in detections:
            for detection in detection_layer:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.4 and classes[class_id] == "sports ball":
                    if confidence > max_confidence:
                        max_confidence = confidence
                        center_x = int(detection[0] * frame_width)
                        center_y = int(detection[1] * frame_height)
                        w = int(detection[2] * frame_width)
                        h = int(detection[3] * frame_height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        ball_box = (x, y, w, h, center_x, center_y)

        if ball_box is not None:
            x, y, w, h, cx, cy = ball_box
            cv2.circle(frame, (cx, cy), 15, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            trajectory_points.appendleft((cx, cy))
        
        for i in range(1, len(trajectory_points)):
            if trajectory_points[i - 1] is None or trajectory_points[i] is None:
                continue
            thickness = int(np.sqrt(len(trajectory_points) / float(i + 1)) * 2.5)
            cv2.line(frame, trajectory_points[i - 1], trajectory_points[i], (0, 0, 255), thickness)

        out.write(frame)

    print(f"\nProcessing complete. Output video saved to: {output_path}")
    cap.release()
    out.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    # Ensure you have the necessary libraries:
    # pip install opencv-python numpy gdown requests

    # 1. Download necessary files
    download_files()

    # 2. Load the deep learning model
    net, classes, output_layers = load_yolo_model()

    video_file = "volleyball.mp4"
    output_video_file = "volleyball_tracked.mp4"

    # 3. Perform player count analysis
    count_players(video_file, net, classes, output_layers)

    # 4. Perform ball tracking and create the output video
    track_ball_trajectory(video_file, output_video_file, net, classes, output_layers)

volleyball.mp4 already exists.
yolov3.weights exists but is too small. Deleting and redownloading.

Successfully downloaded yolov3.weights
yolov3.cfg already exists.
coco.names already exists.

Loading YOLO model...


error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/dnn/src/darknet/darknet_io.cpp:991: error: (-213:The function/feature is not implemented) Transpose the weights (except for convolutional) is not implemented in function 'ReadDarknetFromWeightsStream'


In [1]:
import cv2
import numpy as np
import os
import gdown
import requests
from collections import deque

def download_file_from_url(url, filename):
    """A robust function to download a file from a URL with progress."""
    print(f"Downloading {filename} from {url}...")
    try:
        with requests.get(url, stream=True, timeout=30) as r:
            r.raise_for_status()  # Raise an exception for bad status codes
            total_size = int(r.headers.get('content-length', 0))
            block_size = 8192  # 8KB
            
            with open(filename, 'wb') as f:
                # Use a simple progress indicator
                downloaded_size = 0
                for chunk in r.iter_content(chunk_size=block_size):
                    f.write(chunk)
                    downloaded_size += len(chunk)
                    if total_size > 0:
                        progress = downloaded_size / total_size * 100
                        print(f"\r  -> Progress: {progress:.1f}% ({downloaded_size/1024/1024:.1f}MB / {total_size/1024/1024:.1f}MB)", end="")
        print(f"\nSuccessfully downloaded {filename}")
    except requests.exceptions.RequestException as e:
        print(f"\nError downloading {filename}: {e}")
        # Clean up partially downloaded file if it exists
        if os.path.exists(filename):
            os.remove(filename)

def download_files():
    """Downloads the video and YOLO model files if they don't exist."""
    files_to_download = {
        # Video file from Google Drive
        "volleyball.mp4": ("1JLNjXLnRJUWbzbH1V11DjzPrgjYswGDJ", "gdrive"),
        # YOLOv3 model files
        "yolov3.weights": ("https://pjreddie.com/media/files/yolov3.weights", "url"),
        "yolov3.cfg": ("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", "url"),
        "coco.names": ("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", "url")
    }

    for filename, (source, dl_type) in files_to_download.items():
        # Check if file exists and is valid
        file_is_valid = False
        if os.path.exists(filename):
            # For the large weights file, check if its size is reasonable (>200MB)
            if filename == "yolov3.weights":
                if os.path.getsize(filename) > 200 * 1024 * 1024:
                    print(f"{filename} already exists and seems valid.")
                    file_is_valid = True
                else:
                    print(f"{filename} exists but is too small. Deleting and redownloading.")
                    os.remove(filename)
            else:
                 print(f"{filename} already exists.")
                 file_is_valid = True

        # If the file doesn't exist or was invalid, download it
        if not file_is_valid:
            if dl_type == "gdrive":
                print(f"Downloading {filename} from Google Drive...")
                gdown.download(id=source, output=filename, quiet=False)
            else: # 'url'
                download_file_from_url(source, filename)


def load_yolo_model():
    """Loads the YOLOv3 model and class names."""
    print("\nLoading YOLO model...")
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    output_layers = [net.getLayerNames()[i - 1] for i in net.getUnconnectedOutLayers()]
    print("YOLO model loaded successfully.")
    return net, classes, output_layers

def count_players(video_path, net, classes, output_layers):
    """Counts players on each team based on their position in a single frame."""
    print("\n--- Starting Player Count Analysis ---")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video for player counting.")
        return

    # Read a specific frame (e.g., frame 150) where players are visible
    cap.set(cv2.CAP_PROP_POS_FRAMES, 150)
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read a suitable frame for counting.")
        cap.release()
        return

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes = []
    confidences = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "person":
                center_x = int(detection[0] * width)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(detection[1] * height - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    team1_count = 0
    team2_count = 0
    center_line_x = width // 2

    if hasattr(indexes, 'flatten'):
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            if (x + w/2) < center_line_x:
                team1_count += 1
            else:
                team2_count += 1

    print("\n--- Player Count Results ---")
    print(f"Team 1 (left side) players found: {team1_count}")
    print(f"Team 2 (right side) players found: {team2_count}")
    
    cap.release()


def track_ball_trajectory(video_path, output_path, net, classes, output_layers):
    """Tracks the ball, highlights it, and draws its trajectory."""
    print("\n--- Starting Ball Tracking and Trajectory Visualization ---")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video for tracking.")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    trajectory_points = deque(maxlen=32)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        print(f"Processing frame {frame_count + 1}/{total_frames}...", end='\r')

        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        detections = net.forward(output_layers)

        ball_box = None
        max_confidence = 0.0

        for detection_layer in detections:
            for detection in detection_layer:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.4 and classes[class_id] == "sports ball":
                    if confidence > max_confidence:
                        max_confidence = confidence
                        center_x = int(detection[0] * frame_width)
                        center_y = int(detection[1] * frame_height)
                        w = int(detection[2] * frame_width)
                        h = int(detection[3] * frame_height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        ball_box = (x, y, w, h, center_x, center_y)

        if ball_box is not None:
            x, y, w, h, cx, cy = ball_box
            cv2.circle(frame, (cx, cy), 15, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            trajectory_points.appendleft((cx, cy))
        
        for i in range(1, len(trajectory_points)):
            if trajectory_points[i - 1] is None or trajectory_points[i] is None:
                continue
            thickness = int(np.sqrt(len(trajectory_points) / float(i + 1)) * 2.5)
            cv2.line(frame, trajectory_points[i - 1], trajectory_points[i], (0, 0, 255), thickness)

        out.write(frame)

    print(f"\nProcessing complete. Output video saved to: {output_path}")
    cap.release()
    out.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    # Ensure you have the necessary libraries:
    # pip install opencv-python numpy gdown requests

    # 1. Download necessary files
    download_files()

    # 2. Load the deep learning model
    net, classes, output_layers = load_yolo_model()

    video_file = "volleyball.mp4"
    output_video_file = "volleyball_tracked.mp4"

    # 3. Perform player count analysis
    count_players(video_file, net, classes, output_layers)

    # 4. Perform ball tracking and create the output video
    track_ball_trajectory(video_file, output_video_file, net, classes, output_layers)

volleyball.mp4 already exists.
yolov3.weights exists but is too small. Deleting and redownloading.

Successfully downloaded yolov3.weights
yolov3.cfg already exists.
coco.names already exists.

Loading YOLO model...


error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/dnn/src/darknet/darknet_io.cpp:991: error: (-213:The function/feature is not implemented) Transpose the weights (except for convolutional) is not implemented in function 'ReadDarknetFromWeightsStream'
